In [1]:
#Importing necessary packages
import numpy as np  
import pandas as pd 
import re           
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords   
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
import warnings
import tensorflow as tf
import datetime
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")


Using TensorFlow backend.


In [2]:
#Using GPU on Colab
tf.test.gpu_device_name()

'/device:GPU:0'

In [3]:
#Our data is on Google Drive. So, Google drive has to be mounted in Colab.
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
df = pd.read_excel('/content/drive/My Drive/news_summary_more.xlsx',encoding='latin-1')

In [5]:
df.head()

,headlines,text
0,upGrad learner switches to career in ML & Al with 90% salary hike,"Saurav Kant, an alumnus of upGrad and IIIT-B's PG Program in Machine learning and Artificial Intelligence, was a Sr Systems Engineer at Infosys with almost 5 years of work experience. The program ..."
1,Delhi techie wins free food from Swiggy for one year on CRED,"Kunal Shah's credit card bill payment platform, CRED, gave users a chance to win free food from Swiggy for one year. Pranav Kaushik, a Delhi techie, bagged this reward after spending 2000 CRED coi..."
2,New Zealand end Rohit Sharma-led India's 12-match winning streak,New Zealand defeated India by 8 wickets in the fourth ODI at Hamilton on Thursday to win their first match of the five-match ODI series. India lost an international match under Rohit Sharma's capt...
3,Aegon life iTerm insurance plan helps customers save tax,"With Aegon Life iTerm Insurance plan, customers can enjoy tax benefits on your premiums paid and save up to Ã¢Â‚Â¹46,800^ on taxes. The plan provides life cover up to the age of 100 years. Also, c..."
4,"Have known Hirani for yrs, what if MeToo claims are not true: Sonam","Speaking about the sexual harassment allegations against Rajkumar Hirani, Sonam Kapoor said, ""I've known Hirani for many years...What if it's not true, the [#MeToo] movement will get derailed."" ""I..."


In [6]:
len(df)

98400

### Data pre processing

In [7]:
#Creating empty dataframe to store  all the articles and corresponding headlines for first 95000 datapoints.
data = pd.DataFrame()

In [8]:
data

""


In [9]:
data=df.iloc[:95000,:]

In [10]:
data.head()

,headlines,text
0,upGrad learner switches to career in ML & Al with 90% salary hike,"Saurav Kant, an alumnus of upGrad and IIIT-B's PG Program in Machine learning and Artificial Intelligence, was a Sr Systems Engineer at Infosys with almost 5 years of work experience. The program ..."
1,Delhi techie wins free food from Swiggy for one year on CRED,"Kunal Shah's credit card bill payment platform, CRED, gave users a chance to win free food from Swiggy for one year. Pranav Kaushik, a Delhi techie, bagged this reward after spending 2000 CRED coi..."
2,New Zealand end Rohit Sharma-led India's 12-match winning streak,New Zealand defeated India by 8 wickets in the fourth ODI at Hamilton on Thursday to win their first match of the five-match ODI series. India lost an international match under Rohit Sharma's capt...
3,Aegon life iTerm insurance plan helps customers save tax,"With Aegon Life iTerm Insurance plan, customers can enjoy tax benefits on your premiums paid and save up to Ã¢Â‚Â¹46,800^ on taxes. The plan provides life cover up to the age of 100 years. Also, c..."
4,"Have known Hirani for yrs, what if MeToo claims are not true: Sonam","Speaking about the sexual harassment allegations against Rajkumar Hirani, Sonam Kapoor said, ""I've known Hirani for many years...What if it's not true, the [#MeToo] movement will get derailed."" ""I..."


In [11]:
len(data)

95000

In [13]:
#For some of the datapoints the headlines or article is missing or is corrupted. Hence in this step we will be dropping 
#such data points.
a=[]
b=[]
for i in range(len(data['headlines'])):
  if type(data['headlines'][i])!= str:
    a.append(i)
  if type(data['headlines'][i]) is datetime.datetime:
    b.append(i)
  


In [14]:
b

[]

In [21]:
a

[21133, 33958, 40548, 42934, 74782, 91242]

In [22]:
data.drop([data.index[21133],data.index[33958],data.index[40548],data.index[74782],data.index[91242],data.index[42934]],inplace=True)

In [23]:
len(data)

99994

In [24]:
len(a)

6

In [17]:
contraction = { 
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I had",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"i'm":"i am",
"i've": "i have",
"i'd": "i had",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so is",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they had",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}

In [18]:
# for contraction
def contract(text):
    description = []
    text = text.split()
    for word in text:
        if word in contraction:
            word = contraction[word]
            description.append(word)
        description.append(word)
    return " ".join(description)
                
# Here we are removing all non alpha numeric characters.
def clean(text):
    text = text.lower()
    text = contract(text)
    text = re.sub('[^A-Za-z0-9]', r' ',text)   
    
    return text

In [19]:
cleaned_text = data["text"].apply(lambda x: clean(x))

In [20]:
cleaned_summary = data['headlines'].apply(lambda x: clean(x))

In [21]:
data['text'] = cleaned_text

In [22]:
data['headlines'] = cleaned_summary

In [23]:
data.head()

,headlines,text
0,upgrad learner switches to career in ml al with 90 salary hike,saurav kant an alumnus of upgrad and iiit b s pg program in machine learning and artificial intelligence was a sr systems engineer at infosys with almost 5 years of work experience the program ...
1,delhi techie wins free food from swiggy for one year on cred,kunal shah s credit card bill payment platform cred gave users a chance to win free food from swiggy for one year pranav kaushik a delhi techie bagged this reward after spending 2000 cred coi...
2,new zealand end rohit sharma led india s 12 match winning streak,new zealand defeated india by 8 wickets in the fourth odi at hamilton on thursday to win their first match of the five match odi series india lost an international match under rohit sharma s capt...
3,aegon life iterm insurance plan helps customers save tax,with aegon life iterm insurance plan customers can enjoy tax benefits on your premiums paid and save up to 46 800 on taxes the plan provides life cover up to the age of 100 years also c...
4,have known hirani for yrs what if metoo claims are not true sonam,speaking about the sexual harassment allegations against rajkumar hirani sonam kapoor said i ve known hirani for many years what if it is it is not true the metoo movement will get derail...


### Tokenizing the data

In [25]:
max_len_text = 0
for i in data['text']:
  l = len(i.split())
  if l>max_len_text:
    max_len_text = l


In [26]:
max_len_text

82

In [27]:
max_len_sum = 0
for i in data['headlines']:
  l = len(i.split())
  if l>max_len_sum:
    max_len_sum = l


In [28]:
max_len_sum

22

In [29]:
#Adding start and end token at beginning and end of each headline.
data['headlines'] = data['headlines'].apply(lambda x:'S_ '+x+' _E')

In [30]:
#Tokenizing the articles and headlines.
t1 = Tokenizer()
X=data['text']
t1.fit_on_texts(list(X))
X = t1.texts_to_sequences(X)
X = pad_sequences(X,maxlen=max_len_text,padding='post')

In [31]:
#Adding 2 to the maximum length of headlines for beginning and ending token.
max_len_sum = max_len_sum+2

In [32]:
t2 = Tokenizer()
y=data['headlines']
t2.fit_on_texts(list(y))
y = t2.texts_to_sequences(y)
y = pad_sequences(y,maxlen=max_len_sum,padding='post')

In [33]:
X[:2,:]

array([[25484,  6849,    26,  8010,     5,  9736,     6, 10908,  1529,
            8,  9737,  1941,     4,  1885,  2288,     6,  1566,   780,
           14,     3, 13550,  1888,  2489,    25,  1420,    17,   992,
          100,    76,     5,   245,  1447,     1,  1941,     6,  9736,
            8,  4917,  2764,   769,   487,  1047,    62,  5497,     2,
            3,   244,  3861,    25,  2432,  2041,    17,  1175,  1642,
         2458,  9736,     8,   331,   458,  2288,     9,  2349,   130,
          119,  9491,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0],
       [ 4883,   347,     8,  1513,  1225,   670,  1660,   567, 23799,
          755,   188,     3,  2582,     2,   333,   498,   460,    23,
         6510,    10,    64,    32, 15079,  8983,     3,    83, 21242,
         4189,    39,  3120,    21,  2901,  2868, 23799,  4989,   188,
          206,    64, 23799,  3974,   219,  3189,     5,   67

In [34]:
#Tokenized articles.
X.shape

(95000, 82)

In [35]:
#Tokenized headlines.
y.shape

(95000, 24)

In [44]:
y[0]

array([ 1,  6, 74, 26, 33,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
      dtype=int32)

In [37]:
X_voc_size = (len(t1.word_index)+1)
y_voc_size = (len(t2.word_index)+1)

### Creating and training our model

In [1]:
#We have defined our custom attention layer.I have used the General Score function for attention
def Attention(enc_op,dec_op):
  W = Dense(420)#This is a trainable weight matrix.
  e = tf.matmul(dec_op,W(enc_op),transpose_b=True)
  attention = tf.nn.softmax(e, axis=2)
  context_vec = tf.matmul(attention,enc_op)

  return context_vec


In [39]:

def Encoder(en_input):
  # en_input = np.resize(en_input,(None,82,32))
  enc_emb = Embedding(X_voc_size,32,input_length=82,trainable=True)(en_input)
  enc_lstm1 = Bidirectional(LSTM(210,return_sequences = True,return_state=True,batch_input_shape=(None,80,32)))
  enc_op1, forward_h1, forward_c1, backward_h1, backward_c1 = enc_lstm1(enc_emb)

  enc_lstm2 = Bidirectional(LSTM(210,return_sequences = True,return_state=True))
  enc_op2, forward_h2, forward_c2, backward_h2, backward_c2 = enc_lstm2(enc_op1)

  enc_h = Concatenate()([forward_h2, backward_h2])
  enc_c = Concatenate()([forward_c2, backward_c2])


  
  return enc_op2,enc_h,enc_c





In [40]:
def Decoder(dec_ip,enc_op,state_h,state_c):
  # dec_ip = np.resize(dec_ip,(None,23,32))
  dec_emb = Embedding(y_voc_size,32,input_length=23,trainable=True)(dec_ip)

  lstm = LSTM(420,return_sequences=True,return_state=True,batch_input_shape=(None,10,32))
  dec_op,_,_ = lstm(dec_emb,initial_state=[state_h,state_c])

  context_vec = Attention(enc_op,dec_op)

  concat = Concatenate(axis=-1)([dec_op, context_vec])
  return concat


In [41]:
def seq2seq():
  en_ip = Input((82))
  dec_ip = Input((23))

  enc_op,state_h,state_c=Encoder(en_ip)

  concat = Decoder(dec_ip,enc_op,state_h,state_c)

  dense = Dense(y_voc_size,activation='softmax')
  output = dense(concat)


  model = Model([en_ip, dec_ip], output)
  model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

  return model





In [42]:
model = seq2seq()

In [46]:
from sklearn.model_selection import train_test_split
X_train,X_val,y_train,y_val=train_test_split(X,y,test_size=0.1,random_state=0,shuffle=True)

In [44]:
callbacks = [EarlyStopping(monitor='val_loss', patience=10, verbose=1, min_delta = 0.23,mode='min',),
             ModelCheckpoint('model_speed_pred.h5', monitor = 'val_loss',save_best_only = True, mode = 'min', verbose = 1,save_weights_only = True)]

In [47]:
model.fit([X_train,y_train[:,:-1]], y_train.reshape(y_train.shape[0],y_train.shape[1], 1)[:,1:] ,epochs=50,callbacks=callbacks,batch_size=212, validation_data=([X_val,y_val[:,:-1]], y_val.reshape(y_val.shape[0],y_val.shape[1], 1)[:,1:]))

Epoch 1/50
404/404 [==============================] - ETA: 0s - loss: 3.6265
Epoch 00001: val_loss improved from inf to 3.37401, saving model to model_speed_pred.h5
404/404 [==============================] - 134s 331ms/step - loss: 3.6265 - val_loss: 3.3740
Epoch 2/50
404/404 [==============================] - ETA: 0s - loss: 3.2607
Epoch 00002: val_loss improved from 3.37401 to 3.18783, saving model to model_speed_pred.h5
404/404 [==============================] - 133s 328ms/step - loss: 3.2607 - val_loss: 3.1878
Epoch 3/50
404/404 [==============================] - ETA: 0s - loss: 3.0688
Epoch 00003: val_loss improved from 3.18783 to 3.06189, saving model to model_speed_pred.h5
404/404 [==============================] - 133s 328ms/step - loss: 3.0688 - val_loss: 3.0619
Epoch 4/50
404/404 [==============================] - ETA: 0s - loss: 2.9259
Epoch 00004: val_loss improved from 3.06189 to 2.93272, saving model to model_speed_pred.h5
404/404 [==============================] - 133s 3

In [146]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 81)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 81, 32)       1107072     input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional (Bidirectional)   [(None, 81, 420), (N 408240      embedding[0][0]                  
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 22)]         0                                            
______________________________________________________________________________________________

In [147]:
y_voc_size

16152

### Testing the model

In [48]:
def inference():
  i = np.random.randint(16000,98420)
  en_ip1 = df['text'][i]
  en_ip = t1.texts_to_sequences([en_ip1])
  en_ip = np.array(en_ip)

  dec_ip =  np.array([[t2.word_index['s']]])
  op = model.predict([en_ip,dec_ip],verbose=0)
  dec_ip1=[]
  output=[]

  while True:
#     op = model.predict([en_ip,dec_ip],verbose=0)
    dec_ip = tf.expand_dims(tf.argmax(op,axis=2),0)
    dec_ip1.append(dec_ip)
    op = model.predict([en_ip,dec_ip1],verbose=0)
    # a = tf.argmax(op,axis=2)
    # a = int(a)
    # print(a)
    # dec_ip = np.array(dec_ip)
    output.append(t2.index_word[dec_ip.numpy()[0][0][0]])
    
    if output[-1] == 'e' or len(output) >= 23:
      break
 
  print(en_ip1)
  print('Actual', df['headlines'][i])
  print('Predicted',' '.join(output))
  
   



In [50]:
inference()

tripura governor tathagata roy on tuesday tweeted   how one wishes gandhiji had anointed sardar vallabhbhai as the first prime minister of india  there would have been no kashmir problem   praising patel on his 142nd birth anniversary  roy said the  mahapurush  had welded india into a single country  he also flagged off the run for unity marathon at agartala s vivekananda maidan 
Actual S_ had gandhi made patel pm  j k would not would not be issue  tripura guv _E
Predicted may not not not not not not not not not not not not not not not not not not not not not not


In [118]:
  i = np.random.randint(16000,98420)
  en_ip = df['text'][i]
  en_ip = t1.texts_to_sequences(en_ip)

In [119]:
type(en_ip)

list

In [128]:
en_ip=np.array(en_ip)
en_ip.shape

(362,)

In [124]:
dec_ip =  np.array([[t2.word_index['s']]])

In [125]:
dec_ip

array([[1]])

In [126]:
type(dec_ip)

numpy.ndarray

In [51]:
i = np.random.randint(1,15000)
en_ip1 = df['text'][i]
en_ip = t1.texts_to_sequences([en_ip1])
en_ip = np.array(en_ip)

dec_ip1= df['headlines'][i]
dec_ip = t1.texts_to_sequences([dec_ip1])
dec_ip = np.array(dec_ip)
dec_ip = dec_ip.reshape(1,dec_ip.shape[1], 1)

pred = model.predict([en_ip,dec_ip])
pred = tf.argmax(pred,axis=-1)

# print


In [52]:
pred

<tf.Tensor: shape=(1, 14), dtype=int64, numpy=
array([[  11,    7, 2556,   80,    7,    7, 2199, 2199, 1700, 2273,    2,
           2,    2,    2]])>

In [53]:
en_ip1

'actress swara bhasker has said a person s  sense of sanctity  does not lie in the body   shame is nothing  there is an integrity to our soul  even if our bodies are violated nobody can take our integrity from us   she added  swara further said   my learning for every girl out there is  do not subscribe to this notion of shame  '

In [54]:
dec_ip1

'S_ your sense of sanctity does not lie in your body  swara bhasker _E'